# Developing JuliaFEM

Author(s): Jukka Aho

**Abstract**: General developer notes.

In [1]:
using Logging
using FactCheck
Logging.configure(level=DEBUG)

Logger(root,DEBUG,PipeEndpoint(open, 0 bytes waiting),root)

## Developing own element

Finite element definition, from [FEniCS-book](https://bitbucket.org/fenics-project/fenics-book/src/7d3a80e7dda0fc279c7964dc6000d57942f11eb3/fenicsbook.cls?at=master) [Ciarlet, 2002]:

-  the domain $T$ is a bounded, closed subset of $\mathbb{R}^d$ (for $d = 1, 2, 3, \dots$) with nonempty interior and piecewise smooth boundary;
- the space $\mathcal{V} = \mathcal{V}(T)$ is a finite dimensional function space on $T$ of dimension $n$;
- the set of degrees of freedom (nodes) $\mathcal{L} = \{\ell_1, \ell_2,\ldots, \ell_{n}\}$ is a basis for the dual space $\mathcal{V}'$; that is, the space of bounded linear functionals on $\mathcal{V}$.

We extend this definition so that domain $T$ can also be empty.

Minimum requirements for element:
- subclass from Element, if not wanting to implement everything by youself
- define basis and partial derivatives of it, because we need to interpolate over it
- give connectivity information, how this element is connected to other elements
- create proper constructor (see example).

Test the element using ``test_element`` function. It it passes, then element implementation should be fine. As an example, we define 4 node quadrilateral element using linear Lagrange basis. We really don't care much how element is implemented as long it's interface is constructed with some rules. The interface is tested using `test_element` and it also gives information how to fix element if something is missing.

In [2]:
using JuliaFEM: Element, Field, Basis

Here's the implementation for element:

In [3]:
type MyQuad4 <: Element
    connectivity :: Array{Int, 1}
    basis :: Basis
    fields :: Dict{Symbol, Array{Field, 1}}
end

Default constructor takes only connectivity information as input argument. Extra arguments may be passed using `args...`

In [4]:
function MyQuad4(connectivity, args...)
    h = Basis((xi) ->
    [(1-xi[1])*(1-xi[2])/4
     (1+xi[1])*(1-xi[2])/4
     (1+xi[1])*(1+xi[2])/4
     (1-xi[1])*(1+xi[2])/4])
    MyQuad4(connectivity, h, Dict())
end

MyQuad4

Some basic charasteristics like number of basis funcitons and dimension:

In [5]:
JuliaFEM.get_number_of_basis_functions(el::Type{MyQuad4}) = 4
JuliaFEM.get_element_dimension(el::Type{MyQuad4}) = 2

get_element_dimension (generic function with 7 methods)

Next we check that everything is well defined:

In [6]:
using JuliaFEM: test_element
test_element(MyQuad4)

07-Oct 09:28:37:INFO:root:Testing element MyQuad4
07-Oct 09:28:37:INFO:root:number of basis functions in this element: 4
07-Oct 09:28:37:INFO:root:Initializing element
07-Oct 09:28:37:INFO:root:Element dimension: 2
07-Oct 09:28:37:INFO:root:Creating new scalar field JuliaFEM.Field{Array{Int64,1}}(0.0,1,[1,2,3,4])
07-Oct 09:28:37:INFO:root:Pushing field to element.
07-Oct 09:28:38:INFO:root:Interpolating scalar field at [0.0,0.0]
07-Oct 09:28:38:INFO:root:Value: 2.5


PipeEndpoint(open, 0 bytes waiting)

If `test_element` passes, element should be well defined. At least in the sense that it has all necessary things defined ready to be used in JuliaFEM. After building element, one can interpolate things in it. Couple examples:

In [7]:
using JuliaFEM: new_field!, push_field!, interpolate, dinterpolate
el1 = MyQuad4([1, 2, 3, 4])
new_field!(el1, :temperature, Field(0.0, [0.0, 0.0, 0.0, 0.0]))
push_field!(el1, :temperature, Field(1.0, [1.0, 2.0, 3.0, 4.0]))
new_field!(el1, :Geometry, Field(0.0, Vector[[0.0,0.0,0.0], [10.0,0.0,0.0], [10.0,1.0,0.0], [0.0,1.0,0.0]]))
new_field!(el1, "heat coefficient", Field(0.0, 2))
push_field!(el1, "heat coefficient", Field(1.0, 3))

2-element Array{JuliaFEM.Field{T},1}:
 JuliaFEM.Field{Int64}(0.0,1,2)
 JuliaFEM.Field{Int64}(1.0,1,3)

07-Oct 09:28:38:INFO:root:Element MyQuad4 passed tests.


In [8]:
# temperature at the middle poinf of the element, 1/4*(1+2+3+4) at t=0.5
interpolate(el1, :temperature, [0.0, 0.0], 0.5)

1.25

In [9]:
# geometry midpoint of element
interpolate(el1, :Geometry, [0.0, 0.0], -Inf)

3-element Array{Float64,1}:
 5.0
 0.5
 0.0

In [10]:
dinterpolate(el1, :Geometry, [0.0, 0.0], -Inf)

3x2 Array{Float64,2}:
 5.0  0.0
 0.0  0.5
 0.0  0.0

In [11]:
# interpolating scalar -> scalar.
interpolate(el1, "heat coefficient", [0.0, 0.0], 0.5), interpolate(el1, "heat coefficient", [0.0, 0.0], Inf)

([0.625,0.625,0.625,0.625],[0.75,0.75,0.75,0.75])

### Summary of developing own elements

Element itself if not calculating anything but only stores fields and basis functions so that the fields can be interpolated. We will provide command `test_element` which will ensure that everything necessary is defined. While lot of things needs to be defined, by subclassing from `Element` most of these are already defined, thanks to multiple dispatch.

## Developing own formulation

Let's consider a Laplace equation
\begin{align}
\Delta{u} &= 0 && \text{on } \Omega \\
\frac{\partial u}{\partial n} &= g && \text{on } \Gamma_{\mathrm{N}}
\end{align}

Weak form is, find $u\in\mathcal{U}$ such that
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x = \int_{\Gamma_{\mathrm{N}}}g v\,\mathrm{d}s \quad \forall v\in\mathcal{V}.
\end{equation}

Minimum requirements for equation: 
- subclass from Equation, if not want to implement from scratch
- it needs to have lhs and rhs functions
- default constructor takes the element as input argument

Now we have function `test_equation`, which we can use to test that everything is working as expected. 

Again thanks to multiple dispatch, you are free to code your weak form however you want as long as it returns lhs and rhs sides for element dofs. This kind of freedom gives good opportunities to wrap e.g. Fortran code from some other projects. And again we have some suggestions ad following these ideas you get a lot of stuff for free. First we look the left hand side of the equation, that is,
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x
\end{equation}

In [12]:
using JuliaFEM: Equation, IntegrationPoint, Quad4

abstract Heat <: Equation

Our basic data type often looks something like this:

In [13]:
"""
Diffusive heat transfer for 4-node bilinear element.
"""
type DC2D4 <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

We must provide default constructor which takes element as input argument:

In [14]:
function DC2D4(el::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    new_field!(el, :temperature) # assign new field "temperature" to element
    DC2D4(el, integration_points, [])
end

DC2D4

Now the actual implementation for $\int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x$:

In [15]:
using JuliaFEM: get_element, get_dbasisdX, has_lhs, has_rhs

"""
Left hand side defined in integration point
"""
function JuliaFEM.get_lhs(eq::DC2D4, ip, t)
    el = get_element(eq)
    dNdX = get_dbasisdX(el, ip.xi, t)
    fld = el["temperature thermal conductivity"](t)
    hc = sum(el(ip.xi) * fld)
    return dNdX*hc*dNdX'
end
JuliaFEM.has_lhs(eq::DC2D4) = true

has_lhs (generic function with 2 methods)

And that's it. If we want to play with this formulation, we must create element and assign this equation for it:

In [16]:
using JuliaFEM: integrate, integrate_lhs, integrate_rhs
el = Quad4([1, 2, 3, 4])
new_field!(el, :Geometry, Field(0.0, Vector[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]]))
new_field!(el, "temperature thermal conductivity", Field(0.0, 6.0))
eq = DC2D4(el)
integrate_lhs(eq, 1.0)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [17]:
JuliaFEM.has_lhs(eq)

true

If rhs or lhs is not defined, integration returns nothing.

In [18]:
isa(integrate_rhs(eq, 1.0), Void)

true

Next heat flux on boundary:

In [19]:
using JuliaFEM: get_basis, Seg2

"""
Diffusive heat transfer for 2-node linear segment.
"""
type DC2D2 <: Heat
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

function DC2D2(el::Seg2)
    integration_points = [
        IntegrationPoint([0], 2.0)]
    new_field!(el, :temperature)
    DC2D2(el, integration_points, [])
end

"""
Right hand side defined in integration point
"""
function JuliaFEM.get_rhs(eq::DC2D2, ip, t)
    el = get_element(eq)
    ϕ = get_basis(el)
    f = el["temperature flux"]
    return ϕ(ip.xi)*f(t)
end
JuliaFEM.has_rhs(eq::DC2D2) = true

has_rhs (generic function with 2 methods)

In [20]:
el = Seg2([1, 2])
new_field!(el, :Geometry, Field(0.0, Vector[[0.0,0.0], [0.0,1.0]]))
new_field!(el, "temperature flux", Field(0.0, 100.0))
eq = DC2D2(el)
integrate_rhs(eq, 1.0)

2-element Array{Float64,1}:
 50.0
 50.0

## Defining own problem

- takes a set of elements and maps corresponding equations for them
- problem should have all required information in order to be solvable

In [21]:
using JuliaFEM: Problem, get_equation, get_dimension

type PlaneHeatProblem <: Problem
    equations :: Array{Equation, 1}
end
PlaneHeatProblem() = PlaneHeatProblem([])

PlaneHeatProblem

In [22]:
JuliaFEM.get_dimension(pr::Type{PlaneHeatProblem}) = 1
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Quad4}) = DC2D4
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Seg2}) = DC2D2

get_equation (generic function with 3 methods)

Our solution procedure so far is therefore

In [23]:
using JuliaFEM: get_connectivity, set_global_dofs!, get_global_dofs
using JuliaFEM: add_element!, get_equations, get_matrix_dimension

# create elements and add necessary properties like connectivity and geometry
el1 = Quad4([2, 3, 4, 5])
new_field!(el1, :Geometry, Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]))
new_field!(el1, "temperature thermal conductivity", Field(0.0, 6.0))
el2 = Seg2([2, 3])
new_field!(el2, :Geometry, Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]]))
new_field!(el2, "temperature flux", Field(1.0, 600.0))

problem = PlaneHeatProblem()
add_element!(problem, el1)
add_element!(problem, el2)

function JuliaFEM.get_connectivity(pr::Problem)
    conn = Int[]
    for eq in get_equations(pr)
        el = get_element(eq)
        append!(conn, get_connectivity(el))
    end
    conn = unique(conn)
    return conn
end

"""
Calculate global dofs for equations, maybe using some bandwidth
minimizing or fill reducing algorithm
"""
function calculate_global_dofs(pr::Problem)
    conn = get_connectivity(pr)
    dim = get_dimension(typeof(pr))
    ndofs = dim*length(conn)
    Logging.debug("total dofs: $ndofs")

    mconn = maximum(conn)
    gdofs = reshape(collect(1:mconn), dim, mconn)
    dofmap = Dict{Int64, Array{Int64, 1}}()
    for (i, c) in enumerate(conn)
        dofmap[c] = gdofs[:, i]
    end
    return dofmap
end

"""
Assign global dofs for equations.
"""
function assign_global_dofs!(pr::Problem, dofmap)
    for eq in get_equations(pr)
        el = get_element(eq)
        c = get_connectivity(el)
        #gdofs = [dofmap[ci] for ci in c]
        gdofs = Int64[]
        for ci in c
            append!(gdofs, dofmap[ci])
        end
        set_global_dofs!(eq, gdofs)
    end
end

dofmap = calculate_global_dofs(problem)
println("dofmap: $dofmap")
assign_global_dofs!(problem, dofmap)

function get_lhs(pr::Problem, t::Float64)
    I = Int64[]
    J = Int64[]
    V = Float64[]
    dim = get_dimension(typeof(pr))
    for eq in filter(has_lhs, get_equations(pr))
        dofs = get_global_dofs(eq)
        lhs = integrate_lhs(eq, t)
        for (li, i) in enumerate(dofs)
            for (lj, j) in enumerate(dofs)
                push!(I, i)
                push!(J, j)
                push!(V, lhs[li, lj])
            end
        end
    end
    return I, J, V
end

function get_rhs(pr::Problem, t::Float64)
    I = Int64[]
    V = Float64[]
    dim = get_dimension(typeof(pr))
    for eq in filter(has_rhs, get_equations(pr))
        dofs = get_global_dofs(eq)
        rhs = integrate_rhs(eq, t)
        for (li, i) in enumerate(dofs)
            push!(I, i)
            push!(V, rhs[li])
        end
    end
    return I, V
end

t = 1.0
A = sparse(get_lhs(problem, t)...)
b = sparsevec(get_rhs(problem, t)..., size(A, 1))

07-Oct 09:28:41:DEBUG:root:total dofs: 4


dofmap: Dict(4=>[3],2=>[1],3=>[2],5=>[4])


4x1 sparse matrix with 2 Float64 entries:
	[1, 1]  =  300.0
	[2, 1]  =  300.0

In [24]:
fdofs = [1, 2]
A[fdofs, fdofs] \ b[fdofs]

2x1 sparse matrix with 2 Float64 entries:
	[1, 1]  =  100.0
	[2, 1]  =  100.0

We still need to consider Dirichlet boundary conditions:
\begin{align}
u &= u_0 && \text{on } \Gamma_{\mathrm{D}} \\
\end{align}

In [25]:
abstract BoundaryProblem <: Problem

type DirichletProblem <: BoundaryProblem
    equations :: Array{Equation, 1}
end
DirichletProblem() = DirichletProblem([])

DirichletProblem

In [26]:
abstract DirichletBC <: Equation

"""
Dirichlet boundary condition element for 2 node line segment
(plane problems).
"""
type DBC2D2 <: DirichletBC
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
    fieldval :: Function
end
function DBC2D2(el::Seg2)
    integration_points = [
        IntegrationPoint([-sqrt(1/3)], 1.0),
        IntegrationPoint([+sqrt(1/3)], 1.0)]
    new_field!(el, "reaction force")
    fieldval(X, t) = 0.0
    DBC2D2(el, integration_points, [], fieldval)
end

function JuliaFEM.get_lhs(eq::DBC2D2, ip, t)
    el = get_element(eq)
    h = get_basis(el)(ip.xi)
    return h*h'
end

function JuliaFEM.get_rhs(eq::DBC2D2, ip, t)
    el = get_element(eq)
    h = get_basis(el)
    f = eq.fieldval
    X = interpolate(el, :Geometry, ip.xi, t)
    return h(ip.xi)*f(X, t)
end
JuliaFEM.has_lhs(eq::DBC2D2) = true
JuliaFEM.has_rhs(eq::DBC2D2) = true
JuliaFEM.get_dimension(pr::Type{DirichletProblem}) = 1
JuliaFEM.get_equation(pr::Type{DirichletProblem}, el::Type{Seg2}) = DBC2D2

get_equation (generic function with 4 methods)

In [27]:
# create elements and add necessary properties like connectivity and geometry
el3 = Seg2([4, 5])
new_field!(el3, :Geometry, Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]]))

bc1 = DirichletProblem()
#get_equation(typeof(bc1), typeof(el3))
#DBC2D2(el3)
add_element!(bc1, el3)

1-element Array{JuliaFEM.Equation,1}:
 DBC2D2(JuliaFEM.Seg2([4,5],JuliaFEM.Basis(basis,j),Dict(:Geometry=>JuliaFEM.Field[JuliaFEM.Field{Array{Array{T,1},1}}(0.0,1,Array{T,1}[[0.0,0.0],[0.0,1.0]])],symbol("reaction force")=>JuliaFEM.Field[])),[JuliaFEM.IntegrationPoint([-0.5773502691896257],1.0,Dict{Any,Any}()),JuliaFEM.IntegrationPoint([0.5773502691896257],1.0,Dict{Any,Any}())],Int64[],fieldval)

In [28]:
assign_global_dofs!(bc1, dofmap)

# integrate and assembly
t = 1.0
A2 = sparse(get_lhs(bc1, t)...)
b2 = sparsevec(get_rhs(bc1, t)...)

4x1 sparse matrix with 2 Float64 entries:
	[3, 1]  =  0.0
	[4, 1]  =  0.0

Now we have two problems defined, 

In [29]:
full(A)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [30]:
full(b)

4x1 Array{Float64,2}:
 300.0
 300.0
   0.0
   0.0

In [31]:
full(A2)

4x4 Array{Float64,2}:
 0.0  0.0  0.0       0.0     
 0.0  0.0  0.0       0.0     
 0.0  0.0  0.333333  0.166667
 0.0  0.0  0.166667  0.333333

In [32]:
full(b2)

4x1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0

In [33]:
Atot = [A A2; A2 zeros(A2)]

8x8 sparse matrix with 24 Float64 entries:
	[1, 1]  =  4.0
	[2, 1]  =  -1.0
	[3, 1]  =  -2.0
	[4, 1]  =  -1.0
	[1, 2]  =  -1.0
	[2, 2]  =  4.0
	[3, 2]  =  -1.0
	[4, 2]  =  -2.0
	[1, 3]  =  -2.0
	[2, 3]  =  -1.0
	⋮
	[8, 3]  =  0.166667
	[1, 4]  =  -1.0
	[2, 4]  =  -2.0
	[3, 4]  =  -1.0
	[4, 4]  =  4.0
	[7, 4]  =  0.166667
	[8, 4]  =  0.333333
	[3, 7]  =  0.333333
	[4, 7]  =  0.166667
	[3, 8]  =  0.166667
	[4, 8]  =  0.333333

In [34]:
btot = [b; b2]

8x1 sparse matrix with 4 Float64 entries:
	[1, 1]  =  300.0
	[2, 1]  =  300.0
	[7, 1]  =  0.0
	[8, 1]  =  0.0

Problem is that now are total matrix Atot has zero rows which needs to be removed.

In [36]:
full(Atot)

8x8 Array{Float64,2}:
  4.0  -1.0  -2.0       -1.0       0.0  0.0  0.0       0.0     
 -1.0   4.0  -1.0       -2.0       0.0  0.0  0.0       0.0     
 -2.0  -1.0   4.0       -1.0       0.0  0.0  0.333333  0.166667
 -1.0  -2.0  -1.0        4.0       0.0  0.0  0.166667  0.333333
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.333333   0.166667  0.0  0.0  0.0       0.0     
  0.0   0.0   0.166667   0.333333  0.0  0.0  0.0       0.0     

In [37]:
r = unique(rowvals(Atot))
println("Unique rows: $r")
xtot = zeros(btot)
F = lufact(Atot[r,r])
s = full(btot[r])
xtot[r] = F \ s
full(xtot)

Unique rows: [1,2,3,4,7,8]


8x1 Array{Float64,2}:
 100.0
 100.0
   0.0
   0.0
   0.0
   0.0
 600.0
 600.0